1. create a spark dataframe without using a csv reader 
2. Make sure each column is the correct type
3. handle NA values

In [123]:
from pyspark.sql import SQLContext, Row,DataFrame
filename = 'airquality.csv'
rdd = sc.textFile(filename)
# rdd.map(lambda x: if x=='NA')
# rdd.collect()[0].split(',')

rdd2=rdd.map(lambda x: x.split(','))
header = rdd2.first()
rdd3=rdd2.filter(lambda x: x != header)

def infer_type(x):
    if '"' in x:
        x = x.replace('"','')
        
    if '.' in x:
        return float(x)
    elif 'N' in x:
        return None
    else:
        return int(x) 
        
cleaned = rdd3.map(lambda x: [infer_type(a) for a in x])
header2 = [x.replace('"','') for x in header]
df = cleaned.toDF(header2)
df.show()

+---+-----+-------+----+----+-----+---+
|   |Ozone|Solar.R|Wind|Temp|Month|Day|
+---+-----+-------+----+----+-----+---+
|  1|   41|    190| 7.4|  67|    5|  1|
|  2|   36|    118|null|  72|    5|  2|
|  3|   12|    149|12.6|  74|    5|  3|
|  4|   18|    313|11.5|  62|    5|  4|
|  5| null|   null|14.3|  56|    5|  5|
|  6|   28|   null|14.9|  66|    5|  6|
|  7|   23|    299| 8.6|  65|    5|  7|
|  8|   19|     99|13.8|  59|    5|  8|
|  9|    8|     19|20.1|  61|    5|  9|
| 10| null|    194| 8.6|  69|    5| 10|
| 11|    7|   null| 6.9|  74|    5| 11|
| 12|   16|    256| 9.7|  69|    5| 12|
| 13|   11|    290| 9.2|  66|    5| 13|
| 14|   14|    274|10.9|  68|    5| 14|
| 15|   18|     65|13.2|  58|    5| 15|
| 16|   14|    334|11.5|  64|    5| 16|
| 17|   34|    307|null|  66|    5| 17|
| 18|    6|     78|18.4|  57|    5| 18|
| 19|   30|    322|11.5|  68|    5| 19|
| 20|   11|     44| 9.7|  62|    5| 20|
+---+-----+-------+----+----+-----+---+
only showing top 20 rows



## Peter's Code

In [ ]:
from pyspark import SparkContext
sc =SparkContext()
from pyspark.sql import SQLContext, Row
sqlCon = SQLContext(sc)

# Read in text file into rdd called rawdata
rawdata = sc.textFile("/Users/peterchen/Downloads/sampleData2.csv")
# Define header
header = rawdata.first() #extract header
# Remove header
rawdata = rawdata.filter(lambda x:x !=header)
# split by commas
mydata = rawdata.map(lambda line: line.split(","))

# Add schema with data types
myrows = mydata.map(lambda p:Row(age=int(p[0]),purchasePrice=float(p[1]),gender=p[2],productSelected=p[3]))

mytable = sqlCon.createDataFrame(myrows)
mytable.registerTempTable("mytable")

mytable.printSchema()

sqlCon.sql("select * from mytable where age > 40 ").show()
sqlCon.sql("select * from mytable where purchasePrice > .5 ").show()
sqlCon.sql("select * from mytable where gender = 'F' ").show()
sqlCon.sql("select * from mytable where productSelected = 'water' ").show()
sqlCon.sql("select sum(purchasePrice) as TotalPurchasesWater from mytable where productSelected = 'water' ").show()
sqlCon.sql("select sum(purchasePrice) as TotalPurchasesWomen from mytable where gender = 'F' ").show()
sqlCon.sql("select gender,sum(purchasePrice) as TotalPurchasesByGender from mytable group by gender ").show()